In [25]:
from gurobipy import *

In [26]:
price=[4.84,5.05,5.17,4.92,4.87,4.92,4.98,4.88,5.20,5.36] #I190,I-290,I-294,I-355,I-55,I-57,I-80,I-88,I-90,I-94 (in this order, Chicago highway)
Out=[0,3,1,5,3,50,100,75,70,75]#assumed randomly
n=10 #number of gas stations
Fuel={}
Mile=[0,8.6,2.1,22.4,8,440,659,538,455,576]#I190,I-290,I-294,I-355,I-55,I-57,I-80,I-88,I-90,I-94 (in this order)
#Pricei = retail diesel price (per gallon) at truck stop i
#Outi = amount of miles that a vehicle must go outof-
#route (OOR) to reach truck stop i
#Milei = distance (miles) from truck stop i−1 to i (not
#including Outi−1 or Outi)
SF=190
MG= 18 #The average and median fuel economy for full-size pickup trucks are 19.1 and 18 miles per gallon
TC=200
LF=50 # according to expert it is 1/4 of full tank capacity
MP= 50
EF=50
#phi=[.2,.47,.116,1.244,.44, 24.44, 36.611,29.88,25.277,32],#miles/18
Fuel[0]=SF
#SF = amount of fuel (gallons) in tank at origin o
#(starting fuel)
#MG = average fuel consumption rate (miles per gallon
#or MPG) for the whole trip
#TC = vehicle tank capacity (e.g., 200 gallons)
#LF = minimum fuel to be maintained in tank at all
#times (lower bound fuel)
#MP = minimum amount of fuel to purchase at truck
#stops (e.g., 50 gallons)
#EF = required amount of fuel in tank at the final
#destination d (ending fuel)
Mile_d=Mile[n-1]
print(Mile_d)

576


In [27]:
model=Model("refuel_station")

In [28]:
phi,delta={},{}
for i in range(n):
        phi[i]=model.addVar(vtype=GRB.INTEGER)
for i in range(n):
    delta[i]=model.addVar(vtype=GRB.BINARY)
model.update()


In [29]:
if i==0:
  Fuel[i]= SF - (Mile[i] + delta[i]*Out[i])/MG
else:
  for i in range(1,n):
    Fuel[i] = Fuel[i-1] + phi[i-1] - (delta[i-1]*Out[i-1]+Mile[i]+delta[i]*Out[i])/MG
Fuel_d = Fuel[n-2] + phi[n-2] - (delta[n-2]*Out[n-2] + Mile[n-1])/MG # n-2 is used as it is the second last stoppage. However, for Miles, n-1 is used as the distance between truck stop n and the
#destination d

In [30]:
for i in range(n):
    model.addConstr(Fuel[i]>=LF)

In [31]:
model.addConstr(Fuel_d>=EF)

<gurobi.Constr *Awaiting Model Update*>

In [32]:
for i in range(n):
    model.addConstr(phi[i]>=delta[i]*MP)

In [33]:
for i in range(n):
    model.addConstr(phi[i]<=delta[i]*TC)

In [34]:
for i in range(n):
    model.addConstr(Fuel[i]+phi[i]<=TC)

In [35]:
obj=quicksum(price[i]*phi[i]for i in range(n))
model.setObjective(obj)

In [36]:
model.update()
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 41 rows, 20 columns and 247 nonzeros
Model fingerprint: 0x044b5b8c
Variable types: 0 continuous, 20 integer (10 binary)
Coefficient statistics:
  Matrix range     [6e-02, 2e+02]
  Objective range  [5e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+02]
Found heuristic solution: objective 260.0000000
Presolve removed 41 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 244 260 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.440000000000e+02, best bound 2.440000000000e+02, gap 0.0000%
